In [ ]:
%run helpers/InfiniteTabs.ipynb

def create_select(options=[], description='', value=None, multiple=False, callback=None, *args, **kwargs):
    widget_type = widgets.Dropdown if multiple is False else widgets.SelectMultiple
    value = value or None if multiple is False else ()
    select = widget_type(
        options=options,
        description=description,
        value=value,
        *args,
        **kwargs
    )
    if callback:
        select.observe(callback, 'value')
    return select


def create_button(description=None, callback=None, *args, **kwargs):
    button = widgets.Button(
        description=description,
        *args,
        **kwargs
    )
    if callback:
        button.on_click(callback)
    return button


def make_features_selection_form() -> InfiniteTabs:
    global dfloader, dfloader_tabs

    def tab_factory(uid):
        sheet_name: str = None

        def on_sheet_select(e) -> None:
            nonlocal uid, sheet_name, columns_select
            select, value = (e.owner, e.new)
            sheet_name = value
            dfloader_tabs.set_title(uid, sheet_name)
            select.disabled = True
            columns_select.options = dfloader.sheets[sheet_name].columns
            columns_select.rows = len(columns_select.options)

        def on_submit(e) -> None:
            nonlocal columns_type, columns_select, sheet_name
            dfloader.remove_sheet(sheet_name)
            if columns_type.value == 'features':
                dfloader.add_features(*columns_select.value, sheet_name=sheet_name)
            else:
                dfloader.add_targets(*columns_select.value, sheet_name=sheet_name)

        def on_remove(e) -> None:
            nonlocal sheet_name
            dfloader.remove_sheet(sheet_name)
            dfloader_tabs.remove_tab(uid)

        columns_type = widgets.RadioButtons(options=['features', 'targets'], value='features')
        columns_type.observe(on_submit, 'value')

        sheet_select = create_select(dfloader.sheet_names, value=None, callback=on_sheet_select)
        columns_select = create_select([], value=None, multiple=True, callback=on_submit)

        remove_button = widgets.Button(description='Discard sheet', button_style='danger')
        remove_button.on_click(on_remove)
        return (
            widgets.VBox(children=[
                widgets.VBox(children=[
                    widgets.HTML(value='Columns type:'), columns_type,
                    widgets.HTML(value='Sheet:'), sheet_select,
                    widgets.HTML(value='Columns:'), columns_select
                ]),
                remove_button
            ]),
            ''
        )
    if dfloader_tabs is not None:
        dfloader_tabs.close()
    dfloader_tabs = InfiniteTabs(
        factory=tab_factory,
        initial_factory_tab=True,
        factory_text='Add sheet',
#         max_tabs = len(dfloader.sheet_names)
    )
    dfloader_tabs.display()


def parse_uploaded_file_interactive(filename: str, buffer: bytes):
    global dfloader, dataset_config
    dfloader = DataFrameLoader.from_buffer(filename, buffer, date_parser=dataset_config["date_parser"])
    make_features_selection_form()
